In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
# pretty printing of pandas dataframe
pd.set_option('expand_frame_repr', False)  

# Get current price data
def get_current_data(from_sym='BTC', to_sym='USD', exchange=''):
    '''
    Gets bitcoin to dollars api from cryptcompare and the response comes as a json file
    '''
    url = 'https://min-api.cryptocompare.com/data/price'    
    
    parameters = {'fsym': from_sym,
                  'tsyms': to_sym }
    
    if exchange:
        print('exchange: ', exchange)
        parameters['e'] = exchange
        
    # response comes as json
    response = requests.get(url, params=parameters)   
    data = response.json()
    
    return data


get_current_data('BTC','USD','coinbase')

exchange:  coinbase


{'USD': 19515.2}